# Import et variables globales

In [80]:
import os
import scipy.io.wavfile as wav
from sklearn.model_selection import train_test_split
import numpy as np
import json
import pickle

PATH_PROJECT_ROBIN = "/home/robin/Bureau/ETUDES/M2/S2/TAP/"
PATH_PROJECT_QUENTIN = "..."
PATH_PROJECT_PERSONNE = PATH_PROJECT_ROBIN
PATH_RAW_DATASET = "Projet-Traitement-Automatique-de-la-Parole/Dataset/"
GLOBAL_PATH_DATASET = os.path.join(PATH_PROJECT_PERSONNE, PATH_RAW_DATASET)
GLOBAL_PATH_RAW_DATASET = os.path.join(GLOBAL_PATH_DATASET, "speech_commands_v0.02/")
GLOBAL_PATH_BASIC_DATASET = os.path.join(GLOBAL_PATH_DATASET, "Dataset_basique/")

# Create base dataset

In [81]:
sound_dirs = [elem for elem in os.listdir(GLOBAL_PATH_RAW_DATASET) if os.path.isdir(os.path.join(GLOBAL_PATH_RAW_DATASET,elem))]
label2id = {"up" : 0, "down" : 1, "left" : 2, "right" : 3, "unknown" : 4, "silence" : 5}
id2label = {v : k for k,v in label2id.items()}
labels = label2id.keys()
dataset_list = []

In [82]:
for sound_dir in sound_dirs:
    path_to_dir = os.path.join(GLOBAL_PATH_RAW_DATASET, sound_dir)
    if "background" in sound_dir :
        label = "silence"
        for sound_file in os.listdir(path_to_dir):
            path_to_file = os.path.join(path_to_dir, sound_file)
            if not sound_file.endswith('.wav'): 
                continue
            fs, sig = wav.read(path_to_file)
            for i in range(0, len(sig) - 16000, 16000):
                segment = sig[i : i + 16000]
                dataset_list.append((segment, fs, len(sig)/fs, label))
    else :
        if sound_dir in labels :
            label = sound_dir
        else :
            label = "unknown"
        for sound_file in os.listdir(path_to_dir):
            path_to_file = os.path.join(path_to_dir, sound_file)
            fs, sig = wav.read(path_to_file)
            duree = len(sig) / fs    
            if duree == 1 :
                dataset_list.append((sig, fs, duree, label))

/tmp/ipykernel_14630/884998079.py:9: WavFileWarning: Chunk (non-data) not understood, skipping it.
  fs, sig = wav.read(path_to_file)


In [ ]:
X = [data[0] for data in dataset_list]
y = [data[3] for data in dataset_list]
metadata = [{"fs": data[1], "duree": data[2]} for data in dataset_list]

y_encoded = [label2id[label] for label in y]

X_train, X_temp, y_train, y_temp, meta_train, meta_temp = train_test_split(
    X, y_encoded, metadata, test_size=0.2, random_state=42, stratify=y_encoded)
X_val, X_test, y_val, y_test, meta_val, meta_test = train_test_split(
    X_temp, y_temp, meta_temp, test_size=0.5, random_state=42, stratify=y_temp)

dataset_dict = {
    "train": {
        "signals": X_train, 
        "labels": y_train, 
        "metadata": meta_train
    },
    "val": {
        "signals": X_val,   
        "labels": y_val,   
        "metadata": meta_val
    },
    "test": {
        "signals": X_test,  
        "labels": y_test,  
        "metadata": meta_test
    },
    "mapping": label2id
}

filename = os.path.join(GLOBAL_PATH_DATASET, "dataset_basique.pkl")
with open(filename, "wb") as f:
    pickle.dump(dataset_dict, f)